In [ ]:
from data_generator import DR_Generator
import os
from models import SMD_Unet
from trainer import Trainer
import tensorflow as tf

In [ ]:

masks = ['HardExudate_Masks', 'Hemohedge_Masks', 'Microaneurysms_Masks', 'SoftExudate_Masks']
mask_dir = '../data/Seg-set'
mask_paths = [os.path.join(mask_dir, mask) for mask in masks]

generator_args = {
  'dir_path':'../data/Seg-set/Original_Images/',
  'mask_path':mask_paths,
  'use_mask':True,
  'img_size':(512, 512),  
  'batch_size':4, # 8로 하면 바로 OOM 뜸
  'dataset':'FGADR', # FGADR or EyePacks
  'is_train':True
}

tr_eyepacks_gen = DR_Generator(start_end_index=(0, 500), **generator_args)
val_eyepacks_gen = DR_Generator(start_end_index=(500, 600), **generator_args)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001) 

model = SMD_Unet(filters=[32, 64, 128, 256, 512])

trainer_args = {
    'model':model,
    'epochs':1,
    'optimizer':optimizer,
    'for_recons':False,
    'alpha':1.0,
    'beta':[1.0, 1.0, 1.0, 1.0]
}
trainer = Trainer(**trainer_args)

trainer.train(train_dataset=tr_eyepacks_gen,
              val_dataset=val_eyepacks_gen
              )